# Code 5-1 to Code 5-15

In [1]:
# Read trainig data

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

t1 = pd.read_csv("bank4.csv",encoding = 'latin1')

In [3]:
t1.head()

,Line,Final intent
0,Hi,greetings
1,Hi. My name is,greetings
2,I am facing issues with my credit card,others
3,please help waive my annual membership,Annual Fee Reversal
4,please reverse my annual charges,Annual Fee Reversal


For each of the intent we have the next response to represent beginning of the workflow to illustrate the concept. This is in fp_qns dataset

In [4]:
fp_qns = pd.read_csv('follow_up_qns_v1.csv')
fp_qns.head()

,cat,count,sent
0,greetings,1525,Hi how can I help you
1,Foreign Travel,700,I understad you want to notify us on your trav...
2,Card Delivery,600,I will quickly check on your card delivery status
3,Report Fraud,600,Please contact XXXX-XXXXX for reporting fraud
4,branch_atm_locator,550,Please provide your address


Distribution of intents

In [5]:
t1["Final intent"].value_counts()

greetings               1525
Foreign Travel           700
Card Delivery            600
Report Fraud             600
branch_atm_locator       550
Track application        450
Credit Limit related     400
Pin related              400
Card Activation          350
Card Cancellation        350
Blocked Card             350
Payment related          300
others                   227
Statement related        150
Annual Fee Reversal      150
Name: Final intent, dtype: int64

We do the following set of operations using regex

1.	Replacing special characters like (“\r”,”\n”) with space
2.	Replacing weblinks with a word that indicates it’s a “url”. Whenever we replace entities such as these we add “_pp”. Hence weblinks get replaced with a token “url_pp”.  This is done in order to indicate it’s a replaced word for later analysis. This distinction also helps the model treat from a word that is naturally that is present in the corpus (“url” for instance) to the one replaced using preprocessing stage (“url_pp”)
3.	Replace dates and mobile numbers
4.	Replace percentages
5.	Replace money values
6.	Replace card numbers and otp
7.	Replace any other number which was not captured till step 6 into “simp_digit_pp”
8.	Given we converted all numbers and special characters so far, we treat any non digit (other than space or “_”) to space


In [7]:
def preproc(newdf):
    newdf.ix[:,"line1"]=newdf.Line.str.lower()
    newdf.ix[:,"line1"] = newdf.line1.str.replace("inr|rupees","rs")
    newdf.ix[:,"line1"] = newdf.line1.str.replace("\r"," ")
    newdf.ix[:,"line1"] = newdf.line1.str.replace("\n"," ")
    newdf.ix[:,"line1"] = newdf.line1.str.replace("[\s]+"," ")


    newdf.ix[:,"line1"] = newdf.line1.str.replace('http[0-9A-Za-z:\/\/\.\?\=]*',' url_pp ')
    newdf.ix[:,"line1"] = newdf.line1.str.replace('[0-9]+\/[0-9]+\/[0-9]+',' date_pp ')
    newdf.ix[:,"line1"] = newdf.line1.str.replace('91[7-9][0-9]{9}', ' mobile_pp ')
    newdf.ix[:,"line1"] = newdf.line1.str.replace('[7-9][0-9]{9}', ' mobile_pp ')

    newdf.ix[:,"line1"] = newdf.line1.str.replace('[0-9]+%', ' digits_percent_pp ')
    newdf.ix[:,"line1"] = newdf.line1.str.replace('[0-9]+percentage', ' digits_percent_pp ')
    newdf.ix[:,"line1"] = newdf.line1.str.replace('[0-9]+th', ' digits_th_pp ')
    newdf.ix[:,"line1"] = newdf.line1.str.replace('rs[., ]*[0-9]+[,.]?[0-9]+[,.]?[0-9]+[,.]?[0-9]+[,.]?',' money_digits_pp ')
    newdf.ix[:,"line1"] = newdf.line1.str.replace('rs[., ]*[0-9]+',' money_digits_small_pp ')


    newdf.ix[:,"line1"] = newdf.line1.str.replace('[0-9]+[x]+[0-9]*',' cardnum_pp ')
    newdf.ix[:,"line1"] = newdf.line1.str.replace('[x]+[0-9]+',' cardnum_pp ')
    newdf.ix[:,"line1"] = newdf.line1.str.replace('[0-9]{4,7}',' simp_digit_otp ')
    newdf.ix[:,"line1"] = newdf.line1.str.replace('[0-9]+',' simp_digit_pp ')

    newdf.ix[:,"line1"] = newdf.line1.str.replace("a/c"," ac_pp ")
    newdf.ix[:,"line1"] = newdf.line1.str.replace('[^a-z _]',' ')

    newdf.ix[:,"line1"] = newdf.line1.str.replace('[\s]+,'," ")
    newdf.ix[:,"line1"] = newdf.line1.str.replace('[^A-Za-z_]+', ' ')
    
    return newdf



In [8]:
t2 = preproc(t1)

We will do a stratified sampling of train and test dataset. Train is used for building the model and test is done for validation. Stratified sampling is used so as to keep the distribution of the 14 intents similar in train and test  

In [79]:
tgt = t2["Final intent"]

from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(test_size=0.1,random_state=42,n_splits=1)

for train_index, test_index in sss.split(t2, tgt):
    x_train, x_test = t2[t2.index.isin(train_index)], t2[t2.index.isin(test_index)]
    y_train, y_test = t2.loc[t2.index.isin(train_index),"Final intent"], t2.loc[t2.index.isin(test_index),"Final intent"]

Trying a standard approach using vectorizer, Selector and loigstic regression

In [80]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(min_df=0.0001,analyzer=u'word',ngram_range=(1, 3),stop_words='english')
tfidf_matrix_tr = tfidf_vectorizer.fit_transform(x_train["line1"])

tfidf_matrix_te = tfidf_vectorizer.transform(x_test["line1"])

x_train2= tfidf_matrix_tr.todense()
x_test2 = tfidf_matrix_te.todense()

In [81]:
from sklearn.feature_selection import SelectPercentile, f_classif
selector = SelectPercentile(f_classif, percentile=40)
selector.fit(x_train2, y_train)
x_train3 = selector.fit_transform(x_train2, y_train)
x_test3 = selector.transform(x_test2)

In [82]:
from sklearn.linear_model import LogisticRegression
clf_log = LogisticRegression()
clf_log.fit(x_train3,y_train)  

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

Measuring accuracy

In [83]:
from sklearn.metrics import accuracy_score
pred=clf_log.predict(x_test3)

print (accuracy_score(y_test, pred))

0.9887482419127989


In [84]:
from sklearn.metrics import f1_score
print (f1_score(y_test, pred,average='micro'))
print (f1_score(y_test, pred,average='macro'))

0.9887482419127989
0.9820022348588211


In [85]:
from sklearn.metrics import confusion_matrix
pd.DataFrame(confusion_matrix(y_test, pred))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,35,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,35,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,35,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,60,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,40,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,70,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,30,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,40,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,60,0,0,0,0,0


Building model using embedding followed by LSTM. We will prepare the input accordingly - it is an array of 2 dimensions - N rows, Length of max words in each sentence. Value of each cell is a number that represents the word

get_max_len gets the maximum length in the corpus. This is used to pad the senteces after tokenization

In [86]:
def get_max_len(list1):
    len_list = [len(i) for i in list1]
    return max(len_list)

We use texts_to_sequences and pad_sequences to get the unique value for the word in each row and then to fill "0"s to match the max sentence length respectively

In [87]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
tokenizer = Tokenizer()

def conv_str_cols(col_tr,col_te):
    
    tokenizer = Tokenizer(num_words=1000)
    tokenizer.fit_on_texts(col_tr)
    
    col_tr1 = tokenizer.texts_to_sequences(col_tr)
    col_te1 = tokenizer.texts_to_sequences(col_te)
    
    
    
    max_len1 = get_max_len(col_tr1)
    
    
    col_tr2 = pad_sequences(col_tr1, maxlen=max_len1, dtype='int32', padding='post')
    col_te2 = pad_sequences(col_te1, maxlen=max_len1, dtype='int32', padding='post')
    
    
    
    return col_tr2,col_te2,tokenizer,max_len1

In [88]:
tr_padded,te_padded, tokenizer,max_len1 = conv_str_cols(x_train["line1"],x_test["line1"])

We have in this array N * L (length with padding). 

In [89]:
tr_padded.shape,te_padded.shape

((6391, 273), (711, 273))

We now convert the “dependent” variable into  one hot encoded encoded format so that we can apply a soft-max classifier at the end.



In [104]:
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train1 = le.fit_transform(y_train)
y_test1 = le.fit_transform(y_test)

y_train2 = to_categorical(y_train1)
y_test2 = to_categorical(y_test1)

In [105]:
classes_num = len(y_train.value_counts())

In [106]:
y_train2

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [99]:
classes_num

15

Our architecture is simple - we embed the input layers and pass that to LSTM. The output of LSTM gets fed to a “time distributed” dense layer. This layer gets in turn fed into a fully connected dense layer with “classes_num” as the number of nodes

For example for an LSTM with 5 hidden nodes and 3 time-step as input, the input to the time distributed layer with node 1 would be 3d - layer (Samples * 3 * 5) and output would be 3d-layer (samples, 3, 1). Basically we are extracting one value per time step from the LSTM sequence. This is further fed into dense layers. For example if we defined a time distributed layer with 5 nodes then the output would be of shape - (samples,3,5). 

In [127]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.layers import LSTM,Flatten,TimeDistributed

model = Sequential()
model.add(LSTM(5, input_shape=(3, 1), return_sequences=True))

model.add(TimeDistributed(Dense(6)))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_43 (LSTM)               (None, 3, 5)              140       
_________________________________________________________________
time_distributed_18 (TimeDis (None, 3, 6)              36        
Total params: 176
Trainable params: 176
Non-trainable params: 0
_________________________________________________________________


Lets now get back to the original classifier code. We are taking an embedded input here and stacking the first input (outputs of all the input timestep) to the second LSTM. The second LSTM with 50 hidden units provides a 3-d input to time distributed layer. Time distributed layer output gets passed to the final dense layer. All the time step nodes (50) share the same weights with each other.  A softmax classification is then performed 

In [119]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.layers import LSTM,Flatten,TimeDistributed

model = Sequential()
model.add(Embedding(1000, 100, input_length=max_len1))
model.add(LSTM(100,return_sequences=True))
model.add(LSTM(50,return_sequences=True))

model.add(TimeDistributed(Dense(50, activation='relu')))
model.add(Flatten())
#model.add(Flatten())
model.add(Dense(classes_num, activation='softmax'))
# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# summarize the model
print(model.summary())
# fit the model
model.fit(tr_padded, y_train2, epochs=10, verbose=2,batch_size=30)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_24 (Embedding)     (None, 273, 100)          100000    
_________________________________________________________________
lstm_37 (LSTM)               (None, 273, 100)          80400     
_________________________________________________________________
lstm_38 (LSTM)               (None, 273, 50)           30200     
_________________________________________________________________
time_distributed_13 (TimeDis (None, 273, 50)           2550      
_________________________________________________________________
flatten_10 (Flatten)         (None, 13650)             0         
_________________________________________________________________
dense_30 (Dense)             (None, 15)                204765    
Total params: 417,915
Trainable params: 417,915
Non-trainable params: 0
_________________________________________________________________
None

Now we get accuracies with respect to our test dataset

In [109]:

pred_mat = model.predict_classes(te_padded)
#pred = model.predict_classes(te_padded)

from sklearn.metrics import accuracy_score

print (accuracy_score(y_test1, pred_mat))

from sklearn.metrics import f1_score

print (f1_score(y_test1, pred_mat,average='micro'))
print (f1_score(y_test1, pred_mat,average='macro'))

0.9985935302390999
0.9985935302390999
0.9983013632525033


In [113]:
from sklearn.metrics import confusion_matrix
pd.DataFrame(confusion_matrix(y_test1, pred_mat))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,35,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,35,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,35,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,60,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,40,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,70,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,30,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,40,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,60,0,0,0,0,0


In [111]:
pd.Series(pred_mat).value_counts()

13    154
6      70
9      60
4      60
12     55
11     45
8      40
5      40
3      35
2      35
1      35
7      30
14     22
10     15
0      15
dtype: int64

We use our “fp_qns” dataset to get the right response for the right intent. First when a new sentence is posted to the bot, we preprocess and then convert to sequence and pad it to the “maxlen”. We then predict and find out the right response from the “fp_qns” dataset. Here is an example

In [57]:
def pred_new_text(txt1):
    print ("customer_text:",txt1)
    
    
    newdf = pd.DataFrame([txt1])
    newdf.columns = ["Line"]
    newdf1 = preproc(newdf)
    

    col_te1 = tokenizer.texts_to_sequences(newdf1["line1"])
    col_te2 = pad_sequences(col_te1, maxlen=max_len1, dtype='int32', padding='post')

    class_pred = le.inverse_transform(model.predict_classes(col_te2))[0]
    
    
    resp = fp_qns.loc[fp_qns.cat==class_pred,"sent"].values[0]
    
    print ("Bot Response:",resp,"\n")
    return 

In [159]:
pred_new_text("want to report card lost")
pred_new_text("good morning")
pred_new_text("where is atm")
pred_new_text("cancel my card")


customer_text: want to report card lost
Bot Response: Please contact XXXX-XXXXX for reporting fraud 

customer_text: good morning
Bot Response: Hi how can I help you 

customer_text: where is atm
Bot Response: Please provide your address 

customer_text: cancel my card
Bot Response: Please contact XXXX-XXXXX for cancellation queries 

